In [1]:
import sys
import numpy as np
import pandas as pd

## Load DMS and store object

In [2]:
sys.path.append('../')

from data_prep import dm_saver, data_manager

dms = dm_saver.DM_Saver.load('../dms')

dml = data_manager.Data_Manager()

dml.dms2dm(dms)

In [6]:
print(dml.data_A.shape)
print(dml.data_B.shape)
print(dml.data_C.shape)

(34061, 47)
(32819, 47)
(26071, 47)


,pv_measurement,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,month,hours,sum_rad:W
0,0.00,2019-06-02 22:00:00,7.700,1.22825,0.000000e+00,0.000000,0.0,280.299988,0.000000,0.000000,...,286.225006,100.000000,40386.476562,3.600,-3.575,-0.500,0.0,1.0,0.066987,0.000000
1,0.00,2019-06-02 23:00:00,7.700,1.22350,0.000000e+00,0.000000,0.0,280.299988,0.000000,0.000000,...,286.899994,100.000000,33770.648438,3.350,-3.350,0.275,0.0,1.0,0.017037,0.000000
2,0.00,2019-06-03 00:00:00,7.875,1.21975,0.000000e+00,0.000000,0.0,280.649994,0.000000,0.000000,...,286.950012,100.000000,13595.500000,3.050,-2.950,0.750,0.0,1.0,0.000000,0.000000
3,0.00,2019-06-03 01:00:00,8.425,1.21800,2.086500e+02,0.750000,0.0,281.674988,0.300000,526.775024,...,286.750000,100.000000,2321.850098,2.725,-2.600,0.875,0.0,1.0,0.017037,0.350000
4,19.36,2019-06-03 02:00:00,8.950,1.21800,3.246815e+04,23.100000,0.0,282.500000,11.975000,22068.949219,...,286.450012,99.224998,11634.799805,2.550,-2.350,0.925,0.0,1.0,0.066987,11.741668
5,251.02,2019-06-03 03:00:00,9.250,1.21650,1.794991e+05,84.375000,0.0,283.049988,45.125000,102788.046875,...,286.475006,94.800003,29848.199219,2.300,-2.200,0.800,0.0,1.0,0.146447,45.266666
6,263.78,2019-06-03 04:00:00,9.525,1.21300,4.781178e+05,186.649994,0.0,283.524994,89.525002,242438.000000,...,286.825012,95.099998,35980.148438,2.325,-2.200,0.750,0.0,1.0,0.250000,106.633331
7,522.72,2019-06-03 05:00:00,9.700,1.20750,8.926679e+05,311.525024,0.0,283.799988,139.000000,411381.906250,...,287.325012,97.150002,37874.523438,2.225,-2.000,0.950,0.0,1.0,0.370590,167.875015
8,904.42,2019-06-03 06:00:00,9.550,1.20500,1.357902e+06,442.750000,0.0,283.600006,167.100006,550960.562500,...,287.450012,99.199997,25470.250000,2.350,-1.750,1.525,0.0,1.0,0.500000,214.558334
9,1238.82,2019-06-03 07:00:00,9.450,1.20500,1.821739e+06,567.299988,0.0,283.524994,217.699997,692633.125000,...,287.325012,100.000000,2655.699951,2.700,-1.175,2.425,0.0,1.0,0.629410,267.533325


## Locating constant pvm y-intervals
Dataset B and C has 'holes' in y_train, where pv_measurement is constant. Therefore we don't want to train on this data and can remove the corresponding rows for both X and Y

In [4]:
y_hole_length = 30
from helpers import find_const_interval

In [9]:
# TAKES A WHILE TO RUN
y_train_a_const_idx, ca = find_const_interval(dml.data_A, 'pv_measurement', y_hole_length)
print('y_train_a anomalies:',ca)
y_train_b_const_idx, cb = find_const_interval(dml.data_B, 'pv_measurement', y_hole_length)
print('y_train_b anomalies:',cb)
y_train_c_const_idx, cc = find_const_interval(dml.data_C, 'pv_measurement', y_hole_length)
print('y_train_c anomalies:',cc)


y_train_a anomalies: 5


### Deleting the rows where pvm is const

In [32]:
train_a = dml.data_A
train_b = dml.data_B
train_c = dml.data_C

In [34]:
train_a.shape

(0, 0)

In [19]:
date_forecast_a_const = train_a.iloc[y_train_a_const_idx]['date_forecast']
date_forecast_a_const_values = date_forecast_a_const.values
train_a = train_a[~train_a['date_forecast'].isin(date_forecast_a_const_values)]

date_forecast_b_const = train_b.iloc[y_train_b_const_idx]['date_forecast']
date_forecast_b_const_values = date_forecast_b_const.values
train_b = train_b[~train_b['date_forecast'].isin(date_forecast_b_const_values)]

date_forecast_c_const = train_c.iloc[y_train_c_const_idx]['date_forecast']
date_forecast_c_const_values = date_forecast_c_const.values
train_c = train_c[~train_c['date_forecast'].isin(date_forecast_c_const_values)]


KeyError: 'date_forecast'

In [7]:

dml.data_A = train_a
dml.data_B = train_b
dml.data_C = train_c

### Mean value

In [4]:
%store dml

Stored 'dml' (Data_Manager)
